# DuckDB Alternative: High-Performance Middle Layer for YAML Shredder

This notebook demonstrates using **DuckDB** instead of SQLite as the middle layer for:
- Loading YAML/JSON into relational tables
- Transforming and querying data
- Generating SQL DDL for multiple database systems (Snowflake, PostgreSQL, MySQL, SQLite)

## Why DuckDB?
- **Faster**: In-memory OLAP engine
- **Better SQL Support**: Full ANSI SQL + extensions
- **Rich Type System**: Better type inference
- **JSON Native**: Excellent JSON/nested data handling
- **Flexible**: In-memory or file-based
- **No dependencies**: Single dependency with no C extensions

## Section 1: Setup and Installation

First, ensure the yaml_shredder package with DuckDB support is ready.

In [ ]:
import json
from pathlib import Path
from pprint import pprint
import tempfile
import shutil

import pandas as pd
import yaml

from yaml_shredder import (
    TableGenerator,
    DDLGenerator,
    StructureAnalyzer,
    DuckDBLoader,
    load_to_duckdb,
)

print("✓ All imports successful")
print(f"✓ DuckDBLoader available: {DuckDBLoader}")

## Section 2: Create Sample YAML Data

Create a complex YAML structure to demonstrate DuckDB's advantages.

In [ ]:
# Create temporary working directory
TEMP_DIR = Path(tempfile.gettempdir()) / "duckdb_shredder_demo"
if TEMP_DIR.exists():
    shutil.rmtree(TEMP_DIR)
TEMP_DIR.mkdir(parents=True, exist_ok=True)

# Sample YAML with complex nested structure
sample_config = {
    "organization": "TechCorp",
    "region": "us-east",
    "clusters": [
        {
            "name": "prod-cluster",
            "environment": "production",
            "nodes": [
                {"id": "node-1", "cpu": "16", "memory": "64GB"},
                {"id": "node-2", "cpu": "16", "memory": "64GB"},
            ],
            "networking": {
                "vpc": "vpc-prod",
                "subnets": ["subnet-1", "subnet-2"],
            },
        },
        {
            "name": "dev-cluster",
            "environment": "development",
            "nodes": [
                {"id": "node-3", "cpu": "4", "memory": "16GB"},
            ],
            "networking": {
                "vpc": "vpc-dev",
                "subnets": ["subnet-3"],
            },
        },
    ],
    "services": {
        "api": {"replicas": 3, "cpu": "2", "memory": "4GB"},
        "database": {"replicas": 1, "cpu": "8", "memory": "32GB"},
        "cache": {"replicas": 2, "cpu": "1", "memory": "2GB"},
    },
}

yaml_file = TEMP_DIR / "sample_config.yaml"
with open(yaml_file, "w") as f:
    yaml.dump(sample_config, f)

print(f"✓ Created sample YAML: {yaml_file}")
print(f"  File size: {yaml_file.stat().st_size} bytes")
print("\nSample structure:")
pprint(sample_config)

## Section 3: Analyze Structure

Understand the nested structure before transformation.

In [ ]:
with open(yaml_file) as f:
    data = yaml.safe_load(f)

analyzer = StructureAnalyzer(max_depth=4)
analysis = analyzer.analyze(data)

print("=" * 70)
print("STRUCTURE ANALYSIS")
print("=" * 70)
analyzer.print_summary(analysis)

## Section 4: Generate Relational Tables

Convert nested YAML to normalized tables.

In [ ]:
table_gen = TableGenerator(max_depth=None)  # Full flattening
tables = table_gen.generate_tables(data, root_table_name="CONFIG", source_file=yaml_file)

print("\n" + "=" * 70)
print("GENERATED TABLES")
print("=" * 70)
table_gen.print_summary()

print("\nTable Schemas:")
for table_name, df in tables.items():
    print(f"\n📊 {table_name}")
    print(f"   Rows: {len(df)}, Columns: {len(df.columns)}")
    print(f"   Schema: {list(df.columns)}")

## Section 5: DuckDB as Middle Layer

Load tables into DuckDB for fast querying and transformation.

In [ ]:
# Option 1: File-based DuckDB
db_path = TEMP_DIR / "config.duckdb"
loader = DuckDBLoader(db_path)
loader.connect()
loader.load_tables(tables, if_exists="replace", create_indexes=True)
loader.print_summary()

print(f"\n✓ Database file: {db_path}")
if db_path.exists():
    print(f"  File size: {db_path.stat().st_size / 1024:.2f} KB")

## Section 6: DuckDB Advanced Queries

Demonstrate DuckDB's powerful SQL capabilities.

In [ ]:
print("=" * 70)
print("DUCKDB QUERY EXAMPLES")
print("=" * 70)

# Example 1: List all tables
print("\n1️⃣ Tables in database:")
tables_list = loader.list_tables()
for table in tables_list:
    print(f"   - {table}")

# Example 2: Complex aggregation query
if "CLUSTERS" in tables_list:
    print("\n2️⃣ Cluster statistics:")
    result = loader.query('SELECT environment, COUNT(*) as count FROM "CLUSTERS" GROUP BY environment')
    print(result.to_string(index=False))

# Example 3: Schema inspection
print("\n3️⃣ SAMPLE_CONFIG table schema:")
schema = loader.get_table_info("SAMPLE_CONFIG")
print(schema.to_string(index=False))

## Section 7: Generate DDL for Multiple Databases

Use DuckDB to generate schemas for different database systems.

In [ ]:
# Generate DDL for Snowflake
ddl_gen_sf = DDLGenerator(dialect="snowflake")
ddl_snowflake = ddl_gen_sf.generate_ddl(tables, table_gen.relationships)

print("=" * 70)
print("SQL DDL - SNOWFLAKE")
print("=" * 70)
for table_name, sql in list(ddl_snowflake.items())[:2]:  # Show first 2 tables
    print(f"\n-- Table: {table_name}")
    print(sql[:200] + "..." if len(sql) > 200 else sql)

# Generate DDL for PostgreSQL
ddl_gen_pg = DDLGenerator(dialect="postgresql")
ddl_postgres = ddl_gen_pg.generate_ddl(tables, table_gen.relationships)

print("\n" + "=" * 70)
print("SQL DDL - POSTGRESQL")
print("=" * 70)
for table_name, sql in list(ddl_postgres.items())[:2]:  # Show first 2 tables
    print(f"\n-- Table: {table_name}")
    print(sql[:200] + "..." if len(sql) > 200 else sql)

## Section 8: In-Memory DuckDB

Demonstrate DuckDB's in-memory mode for ephemeral data processing.

In [ ]:
# Create in-memory database (no file)
print("Creating in-memory DuckDB database...\n")
memory_loader = DuckDBLoader(db_path=None)  # None = in-memory
memory_loader.connect()
memory_loader.load_tables(tables, if_exists="replace")
memory_loader.print_summary()

print("\n✓ In-memory database is perfect for:")
print("  - Temporary data transformations")
print("  - Unit testing")
print("  - Rapid prototyping")
print("  - Avoiding file I/O overhead")

## Section 9: Performance Comparison

Compare DuckDB vs SQLite performance characteristics.

In [ ]:
print("=" * 70)
print("DUCKDB vs SQLITE COMPARISON")
print("=" * 70)

comparison = """
╔════════════════════╦═════════════════╦════════════════════╗
║ Feature            ║ DuckDB          ║ SQLite             ║
╠════════════════════╬═════════════════╬════════════════════╣
║ Query Speed        ║ Very Fast ⚡⚡⚡ ║ Moderate ⚡⚡     ║
║ Type System        ║ Rich & Smart    ║ Basic              ║
║ JSON Support       ║ Excellent       ║ Limited            ║
║ Memory Efficiency  ║ Excellent       ║ Good               ║
║ Aggregations       ║ Very Fast       ║ Moderate           ║
║ OLAP Workloads     ║ Optimized       ║ Not optimized      ║
║ In-Memory Mode     ║ :memory:        ║ :memory:           ║
║ Ease of Use        ║ Simple          ║ Very Simple        ║
╚════════════════════╩═════════════════╩════════════════════╝
"""

print(comparison)

print("\n✓ BEST FOR DuckDB:")
print("  - Analytical queries on structured data")
print("  - JSON/nested data transformation")
print("  - Complex aggregations and joins")
print("  - High-volume data processing")
print("  - In-memory OLAP workloads")

print("\n✓ BEST FOR SQLite:")
print("  - Simple key-value storage")
print("  - Lightweight embedded databases")
print("  - Low-latency writes")

## Section 10: Summary - Why DuckDB is the Better Choice

For the YAML Shredder use case, DuckDB is superior as a middle layer.

In [ ]:
summary = """
==========================================================================
WHY DUCKDB IS THE IDEAL MIDDLE LAYER FOR YAML SHREDDER
==========================================================================

✓ ADVANTAGES FOR DATA TRANSFORMATION:
  1. Native JSON support → Perfect for YAML/JSON data
  2. Type inference → Automatically detects optimal types
  3. Fast aggregations → Analyze nested data quickly
  4. Rich functions → JSON extraction, array operations

✓ ADVANTAGES FOR DDL GENERATION:
  1. Better schema inspection → More accurate type mapping
  2. Complex queries → Can derive constraints from data
  3. Better null handling → Proper NOT NULL detection
  4. Index optimization → Suggest key columns

✓ ADVANTAGES FOR DEPLOYMENT:
  1. File-based → Can be version controlled
  2. In-memory → No disk overhead during processing
  3. Small footprint → Single .duckdb file
  4. Cross-platform → Works everywhere Python works

✓ PYTHON INTEGRATION:
  1. Seamless pandas support → df.to_duckdb(), query().df()
  2. Simple API → Similar to pandas/SQLite
  3. No dependencies → Pure Python (with native library)
  4. Type hints → Full Python 3.13+ support

==========================================================================
RECOMMENDED ARCHITECTURE
==========================================================================

YAML File
    ↓
Parse with PyYAML
    ↓
Table Generation (TableGenerator)
    ↓
DuckDB ← ← ← [THIS IS YOUR MIDDLE LAYER] ← ← ←
    ↓
Schema Inference & Analysis
    ↓
DDL Generation (for Snowflake/PostgreSQL/MySQL/SQLite)
    ↓
Deploy to Target Database

==========================================================================
"""

print(summary)

print("\n✓ Files created in this demo:")
for file in sorted(TEMP_DIR.rglob("*")):
    if file.is_file():
        rel_path = file.relative_to(TEMP_DIR)
        size = file.stat().st_size
        print(f"  {rel_path} ({size:,} bytes)")

print(f"\n✓ Working directory: {TEMP_DIR}")